## パッケージのインストール

一度だけ実行すれば良い

In [1]:
!pip install pandas
!pip install biopython
!pip install pymsaviz
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.4 MB/s eta 0:00:00


In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


## Restart session

In [1]:
!conda config --add channels bioconda
!conda install -y blast muscle

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - blast
    - muscle


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blast-2.17.0               |       h66d330f_0        80.9 MB  bioconda
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    certifi-2025.10.5          |     pyhd8ed1ab_0         156 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    curl-8.11.1                |       h332b0f4_0         171 KB  conda-forge
    entrez-direct-24.0         |       he881be0_0        16.3 MB  bioconda
    libasprintf-0.25.1         |       h3f43e3d_1          52 KB  conda-forge
    libgettextpo-0.25.1        |       h5888daf_0         174 KB  conda-forge
    libidn2-2.3.8          

## Blastのデータベース作成

Blast+をあらかじめインストールしてください。Ubuntuなら

sudo apt install -y ncbi-blast+

WindowsやMacは解説のスライドがWebにいくつか上がっています。

Blastのデータベースを追加したい場合は、Fastaファイルを用意して、以下の例のようにデータベースを作成してください。

データベースがタンパク質なら　-dbtype prot　核酸配列なら　-dbtype nucl

https://bi.biopapyrus.jp/seq/blast/makeblastdb.html

In [3]:
import subprocess
import pandas as pd
from pathlib import Path
from Bio import SeqIO
import os
from pymsaviz import MsaViz
import matplotlib.pyplot as plt
from Bio.Align.Applications import MuscleCommandline


## Fastaのインポート

Fastaファイルを追加した場合はここにも追記してください。

画面への出力で、IDの記述例を確認してください。

In [4]:
FASTAs = ["Arabidopsis/Araport11_genes.201606.pep.fasta",
          "Benthamiana/Nbe_v1.1_pep.fa",
          "GMax/Gmax_880_Wm82.a6.v1.protein.fa",
          "Marchantia/MpTak_v6.1r2.protein.fasta",
          "Oryza/Osativa_204_v7.0.protein.fasta",
          "Triticum/Triticum_aestivum.IWGSC.pep.all.fa"
          ]
sequences = []
ids = []
descriptions = []
cnt = 0
for filename in FASTAs:
    for record in SeqIO.parse(filename, "fasta"):
        id_part = record.id
        desc_part = record.description
        seq = record.seq.replace("*", "")
        sequences.append(seq)
        ids.append(id_part)
        descriptions.append(desc_part)
        cnt += 1
        if cnt % 20000 == 0: print(id_part)
    print("\nTotal", len(sequences),"genes\n")

FileNotFoundError: [Errno 2] No such file or directory: 'Arabidopsis/Araport11_genes.201606.pep.fasta'

## 相同性検索　Blast search

query_List_fileで指定しているテキストファイル("GeneList.txt")の中身に遺伝子名番号を入れてくださいAT1G26470.1など

1行に1遺伝子づつです。

**resdirは出力先のフォルダです。中身が空になっていることを確認してください。**

In [ ]:
query_List_file = "GeneList.txt"
resdir = "results"
os.makedirs(resdir, exist_ok=True)



使いたい種のみにdb_Listを書き換えてください。

"Arabidopsis/At_pep", 　　　シロイヌナズナ

"Oryza/Osativa_pep", 　　　イネ

"Benthamiana/Nben_pep", 　ベンサミアナ

"GMax/Glymax_pep.fasta",　ダイズ

"Marchantia/Mp_pep",　　　ゼニゴケ

"Triticum/Ta_pep.pdb"　　　コムギ

In [ ]:
db_List = ["Arabidopsis/At_pep",
           "Oryza/Osativa_pep",
           "Benthamiana/Nben_pep",
           "GMax/Glymax_pep",
           "Marchantia/Mp_pep",
           "Triticum/Ta_pep"
           ]

query_list = []
query_file = open("GeneList.txt", "r")

for query in query_file.readlines():
    if query.strip() == "" :
        continue

    query_list.append(query.strip())


In [ ]:
db_List = ["Arabidopsis/At_pep",
           "Oryza/Osativa_pep",
           "Benthamiana/Nben_pep",
           "GMax/Glymax_pep",
           "Marchantia/Mp_pep",
           "Triticum/Ta_pep"
           ]

query_list = []
query_file = open("GeneList.txt", "r")

for query in query_file.readlines():
    if query.strip() == "" :
        continue

    query_list.append(query.strip())


In [ ]:
query_fasta_name = "query.fasta"
query_fasta = open("query.fasta", "w")

for query in query_list:
    fasta_index = ids.index(query)
    seq = sequences[fasta_index]
    query_fasta.write(">" + query + "\n")
    query_fasta.write(str(seq) + "\n")
query_fasta.close()

In [ ]:
db_name =  " ".join(db_List)
out_file = "blast_results.tsv"
blast_cmd = [
    "blastp",
    "-query", query_fasta_name,
    "-db", db_name,
    "-outfmt","6 qseqid sseqid pident length evalue bitscore",
    "-max_target_seqs", "50", #ここの数字が、1クエリあたりヒット数の上限
    "-num_threads", "4",
    "-out", out_file,
    "-evalue", "1e-5", #e-valueで切る。特定のグループの少し外側まで取って、outgroupeを作ることで、系統樹の根を推定できる。
    "-max_hsps", "1"
]

In [ ]:
print("Running BLAST...")
subprocess.run(blast_cmd, check=True)
print("BLAST finished.")

Running BLAST...
BLAST finished.


"blast_results.tsv"が出力ファイルです。エクセルで開けます。エクセルを立ち上げて、そこにドラックアンドドロップしてください。

"Query", "Subject", "Identity(%)", "Length", "Evalue", "Bitscore"の順番です


In [ ]:
#columns = ["Query", "Subject", "Identity(%)", "Length", "Evalue", "Bitscore"]
infile = open("blast_results.tsv","r")
dat_table = []
querys = []
geneids = []

for line in infile.readlines():
    dat = line.split("\t")
    query = dat[0]
    subject = dat[1]

    geneID_split = subject.split(".")
    if len(geneID_split) >= 2 and len(geneID_split[-1]) <= 2:
        geneid = ".".join(subject.split(".")[0:-1]) #スプライスバリアントの除去
    else:
        geneid = subject

    if len(querys) == 0 or querys[-1] != query:
        geneids = [geneid]
        querys.append(query)
        outfile = open(resdir + "/" + query + ".fasta", "w")
        fasta_index = ids.index(subject)
        seq = sequences[fasta_index]
        outfile.write(">" + subject + "\n")
        outfile.write(str(seq) + "\n")
    else:
        if geneid in geneids: continue #スプライスバリアントを複数残す場合はこの行コメントアウト
        geneids.append(geneid)
        fasta_index = ids.index(subject)
        seq = sequences[fasta_index]
        outfile.write(">" + subject + "\n")
        outfile.write(str(seq) + "\n")
outfile.close()

## Alignment

出力されたファイルは序盤に指定したresdirにfastaが保存されている。

Aligmentの前にファスタファイルを確認して、必要ないものは削除しておく。


In [ ]:
# ここから始めたい場合は、アライメントをかけたいFastaを入れたファイルをフォルダに入れて、resdirとして指定。#を取れば、コメントアウトではなくなる。
# resdir = "results"

In [ ]:

infiles = os.listdir(resdir)
aligned_files = []
print(infiles)

#muscle_exe = "./muscle-linux-x86.v5.3"   #Linux用
muscle_exe = "muscle"    #mac用
#muscle_exe = "./muscle-win64.v5.3.exe" #windows用

for infile in infiles:
    if infile.endswith(".fasta") == False: continue
    if infile.startswith("Aligned_") == True : continue
    in_file = resdir + "/" + infile
    out_file = resdir + "/Aligned_" + infile
    aligned_files.append(resdir + "/Aligned_" + infile)
    subprocess.run([muscle_exe , "-align", in_file, "-output", out_file], check=True)

['Tree', 'Aligned_Aligned_AT1G65480.1.fasta', 'Aligned_AT4G33950.1.tsv', '.DS_Store', 'AT4G33950.1.fasta', 'Aligned_AT1G26470.1.txt', 'Aligned_AT1G65480.1.fasta.png', 'Aligned_AT4G33950.1.fasta', 'AT1G65480.1.fasta', 'Aligned_AT1G26470.1.tsv', 'Aligned_AT1G65480.1.fasta', 'Aligned_AT1G26470.1.fasta.png', 'Aligned_AT4G33950.1.txt', 'Aligned_Aligned_AT4G33950.1.fasta', 'Aligned_AT1G65480.1.txt', 'Aligned_Aligned_AT1G26470.1.fasta', '.ipynb_checkpoints', 'Aligned_AT4G33950.1.fasta.png', 'AT1G26470.1.fasta', 'Aligned_AT1G65480.1.tsv', 'alignment_.png', 'Aligned_AT1G26470.1.fasta']



muscle 5.3.osx64 []  8.6Gb RAM, 4 cores
Built Jul 30 2025 21:35:18
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

[align results/AT4G33950.1.fasta]
Input: 50 seqs, avg length 356, max 399, min 294

00:01 2.2Mb   100.0% Derep 43 uniques, 7 dupes
00:01 2.3Mb  CPU has 4 cores, running 4 threads
00:06 40Mb    100.0% Calc posteriors
00:06 40Mb    100.0% UPGMA5         
00:07 41Mb    100.0% Consistency (1/2)
00:07 42Mb    100.0% Consistency (2/2)
00:07 24Mb    100.0% Refining         

muscle 5.3.osx64 []  8.6Gb RAM, 4 cores
Built Jul 30 2025 21:35:18
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

[align results/AT1G65480.1.fasta]
Input: 50 seqs, avg length 180, max 276, min 173

00:00 2.2Mb   100.0% Derep 47 uniques, 3 dupes
00:00 2.3Mb  CPU has 4 cores, running 4 threads
00:02 19Mb    100.0% Calc posteriors
00:02 19Mb    100.0% UPGMA5         
00:02 24Mb    100.0% Consistency (1/2)
00:02 24Mb    100.0% Consistency (2/2)
00:02 20Mb    100.0% Refining         

m

Alignmentの画像化

 遺伝子数が50個超えてくると時間かかりすぎるので、パスしてください


In [ ]:

for filename in aligned_files:
    mv = MsaViz(filename, wrap_length=None, show_consensus=True, show_count=True, color_scheme="Identity")
    mv.savefig(filename + ".png", dpi=200)

## 系統樹の作成

In [ ]:
tsvs = []
for filename in aligned_files:
    out_file = open(filename.replace(".fasta", ".tsv"), "w")
    tsvs.append(filename.replace(".fasta", ".tsv"))
    ids2 = []
    sequences2 = []
    for record in SeqIO.parse(filename, "fasta"):
        out_file.write(record.id)
        out_file.write("\t")
        out_file.write("\t".join(list(record.seq)) + "\n")


系統樹を作る際に、相同性の低い領域の影響が大きくなるのを防ぐ。

ここでアライメントされた配列から、アライメントが取れていない領域(相同性の低い領域)を消す。tsvファイルをエクセル等で開いて、Deleteしていく

より詳しい方法は、パワポを確認。領域を消すときは、以降のアライメントが崩れないように下から上まで全遺伝子に渡って消す。

例えば、Raf like kinaseの系統樹は、キナーゼドメインのみを用いて作られている。


続いて、tsvをFastaに戻す。スクリプト↓

In [ ]:
os.makedirs(resdir + "/Tree", exist_ok=True)
for tsv in tsvs:
    in_file = open(tsv, "r")
    out_file = open(tsv.replace( ".tsv", ".fasta").replace(resdir, resdir + "/Tree"), "w")
    for line in in_file.readlines():
        data = line.split("\t")
        if len(data) < 4: continue
        out_file.write(">" + data[0] + "\n")
        out_file.write( "".join(data[1:]) + "\n")
    out_file.close()


結果の出力先フォルダのなかにTreeというフォルダがあり、その中に、Fastaが入っている。

そのファイルを使って、MEGA12に読み込ませる。